# Opioid Crisis - Analysis

I want to take a second look at the data from the Opioid Crisis datasheet.

- (link here: https://www.mathmodels.org/Problems/2019/MCM-C/index.html)

Motivation: was part of the MCM 2019.

Data we will work with:
* Drug identification counts in years 2010-2016
* Socio-economic factors collected for five states (Ohio, Kentucky, West Virginia, Virginia, Pennsylvania)

In [ ]:
import numpy as np
import pandas as pd

# drug use data.
df_nflis = pd.read_excel('2018_MCMProblemC_DATA/MCM_NFLIS_Data.xlsx', sheet_name="Data")

# socio-economic data.
df10 = pd.read_csv('2018_MCMProblemC_DATA/ACS_10_5YR_DP02/ACS_10_5YR_DP02_with_ann.csv')
df11 = pd.read_csv('2018_MCMProblemC_DATA/ACS_11_5YR_DP02/ACS_11_5YR_DP02_with_ann.csv')
df12 = pd.read_csv('2018_MCMProblemC_DATA/ACS_12_5YR_DP02/ACS_12_5YR_DP02_with_ann.csv')
df13 = pd.read_csv('2018_MCMProblemC_DATA/ACS_13_5YR_DP02/ACS_13_5YR_DP02_with_ann.csv')
df14 = pd.read_csv('2018_MCMProblemC_DATA/ACS_14_5YR_DP02/ACS_14_5YR_DP02_with_ann.csv')
df15 = pd.read_csv('2018_MCMProblemC_DATA/ACS_15_5YR_DP02/ACS_15_5YR_DP02_with_ann.csv')
df16 = pd.read_csv('2018_MCMProblemC_DATA/ACS_16_5YR_DP02/ACS_16_5YR_DP02_with_ann.csv')

# indexing data.
df10_meta = pd.read_csv('2018_MCMProblemC_DATA/ACS_10_5YR_DP02/ACS_10_5YR_DP02_metadata.csv')
df11_meta = pd.read_csv('2018_MCMProblemC_DATA/ACS_11_5YR_DP02/ACS_11_5YR_DP02_metadata.csv')
df12_meta = pd.read_csv('2018_MCMProblemC_DATA/ACS_12_5YR_DP02/ACS_12_5YR_DP02_metadata.csv')
df13_meta = pd.read_csv('2018_MCMProblemC_DATA/ACS_13_5YR_DP02/ACS_13_5YR_DP02_metadata.csv')
df14_meta = pd.read_csv('2018_MCMProblemC_DATA/ACS_14_5YR_DP02/ACS_14_5YR_DP02_metadata.csv')
df15_meta = pd.read_csv('2018_MCMProblemC_DATA/ACS_15_5YR_DP02/ACS_15_5YR_DP02_metadata.csv')
df16_meta = pd.read_csv('2018_MCMProblemC_DATA/ACS_16_5YR_DP02/ACS_16_5YR_DP02_metadata.csv')

Parts of the data are not available.

# Preprocessing

General plan: iterate over socio-economic data, and append with relevant drug use data.

Feature extraction part:
* Include geography (specifically `GEO.display-label`).
* Exclude margin of error features.
* Exclude columns with `(X)`.
* Exclude non-universal data.

In [ ]:
df10

The function `feature_extract` will extract features as to satisfy the above conditions (save universality).

In [ ]:
from opioid_crisis_lib import feature_extract
df10[feature_extract(df10, df10_meta)]

### Filtering Data with Universal Property

We can only work with properties that are present for all dataframes.

At the same time, the heterogenous nature of labels corresponding to the same descriptor across years prompts us to address that as well, with the following map:
$$\mathrm{descriptor}\mapsto(\mathrm{year}\mapsto\mathrm{label})$$

In [ ]:
from opioid_crisis_lib import feature_index
from opioid_crisis_lib import feature_index2

# ddf = [df10, df11, df12, df13, df14, df15, df16]
# ddf_metadata = [df10_meta, df11_meta, df12_meta, df13_meta, df14_meta, df15_meta, df16_meta]

ddf_yyyy = {
    "2010": df10,
    "2011": df11,
    "2012": df12,
    "2013": df13,
    "2014": df14,
    "2015": df15,
    "2016": df16,
}
ddf_metadata_yyyy = {
    "2010": df10_meta,
    "2011": df11_meta,
    "2012": df12_meta,
    "2013": df13_meta,
    "2014": df14_meta,
    "2015": df15_meta,
    "2016": df16_meta,
}

# f_index = feature_index(ddf, ddf_metadata)
# sorted(f_index)

f_index = feature_index2(ddf_yyyy, ddf_metadata_yyyy)
sorted(f_index)

As we see only the "important" data remain. We've gotten rid of error estimate data as well as inadmissible data.

Given a particular year we want to extract all universal relevant labels. In other words, we want the map:
$$
\left(\mathrm{descriptor}\mapsto(\mathrm{year}\mapsto\mathrm{label})\right)
\mapsto\left(\mathrm{year}\mapsto(\mathrm{labels})\right)
$$

In [ ]:
from opioid_crisis_lib import label_from_feature_index
label_from_feature_index("2012", f_index)

In [ ]:
f_index['Estimate; ANCESTRY - Total population']

We obtain a map from descriptors to corresponding labels. This allows us to go back and forth between description and label.

The above data corresponds to the socio-economic data of a particular county at some specified year. When we go over drug use data, the `YYYY`, `State` and `County` data should sufficiently return the appropriate socio-economic data.

The following short function returns the state and county of a string as separate strings, with state written in initials.

In [ ]:
from opioid_crisis_lib import state_and_county
state_and_county("Adair County, Kentucky")

### Retrieving Geographic Data

In addition, for each county there should be a method to retrieve numerical geographic data:

In [ ]:
df_geo = pd.read_csv('2021_Gaz_counties_national.txt', sep="\t")
from opioid_crisis_lib import locate
locate("ky", "adair", df_geo)

In [ ]:
np.array(df_geo[["USPS", "NAME"]])

### Processing Drug Data

In [ ]:
df_nflis[["YYYY", "State", "COUNTY", "SubstanceName", "DrugReports"]]

Of importance is the type of drugs reported.

In [ ]:
df_nflis["SubstanceName"]

The following gives a survey of distinct drug types (indexed).

In [ ]:
substanceNames = sorted(set(df_nflis["SubstanceName"]))
substanceNames

In [ ]:
print(len(substanceNames))

Convert this to a dictionary so we can map substance use to a particular index.

In [ ]:
substanceNamesDict = {substanceNames[i]:i for i in range(69)}

Given data on `drug reports` and `substance name`, we can construct a vector which indicates extent of a particular drug use in a county.

In [ ]:
from opioid_crisis_lib import drug_matrix
drug_matrix(df_nflis, substanceNamesDict)

Now due to redundancy, a single county may occur multiple times. We want to have one drug vector per county.

In [ ]:
from opioid_crisis_lib import drug_vector
drug_vector("2010", "oh", "adams", df_nflis, substanceNamesDict, identify=True)

### Compiling Data

We can do this:
1. Determine the overall dimension of the sample matrix. (doable)
1. Iterate through socio-economic dataframe rows, through the years 2010-2016,
    1. For each row, gather socio-economic data, AND data which identifies the State, county and year.
    1. Retrieve the geographic location of the county, append.
    1. Retrieve the drug vector of the county, append.
2. Write all appended data into one numpy array.
3. (Optional) Move independent columns (geographical location, socio-economic data) to the beginning, and the drug vector to the end.

**Note**: even with previous filtering, there are still socio-economic sample points with `(X)` terms. These terms are set to 0. (Not the best approach. we'll deal with this later. But I assume that these anomalous rows only take a small portion and are negligible.)

**Note**: turns out that the drug data contains data up to 2017, which is not included in the socio-economic data. However, this new data also contains new drugs, which messes up the data (since the new-drug column is a column of zeros, which messes up pca and all that).

Make sure the data is recoverable and that rows have unique identifiers. (which indicate year, state and county)

**WARNING!!!! The following code will take FOREVER (at least 1 hour)**

In [ ]:
# from opioid_crisis_lib import generate_sample
# sample = generate_sample(ddf_yyyy, ddf_metadata_yyyy, f_index, df_nflis, substanceNamesDict, df_geo)
# sample

In [ ]:
# np.savetxt("please_dont_overwrite.csv", sample, delimiter=",")

In [ ]:
sample_read = pd.read_csv('please_dont_overwrite.csv', header=None)
sample_read

Convert to np.array.

In [ ]:
sample_read_np = sample_read.to_numpy()
sample_read_np

### Extraction of Nonzero Columns (features) and Problematic Rows (sample points)

For a column feature, we need to make sure that the column is nonzero. This is because if a column $c$ is zero, the *standardization* $$\dfrac{c - \mathrm{mean}\,(c)}{\mathrm{std}\,(c)}$$ is undefined (in this case it is `nan`).

Additionally, there still exist sample points with `nan` entries in the first and second rows (when the county location cannot be found). We will also remove those too.

In [ ]:
from opioid_crisis_lib import find_nonzero, keep_rows, keep_cols
nonzero_index, zero_index = find_nonzero(sample_read_np.T)
zero_index

Identify the features these zero indices correspond to.

In [ ]:
geo_features = np.array(["INTPTLAT", "INTPTLONG                                                                                                               "])
features = np.concatenate((["Year"], geo_features, 
                          sorted(f_index),
                          sorted(substanceNamesDict)))

In [ ]:
features[zero_index]

Identify the features the nonzero indices correspond to.

In [ ]:
nonzero_features = features[nonzero_index]
nonzero_features

In [ ]:
nonzero_features.shape

These are drugs which are not included in the 2010-2016 dataframes. We will construct a matrix without these features.

In [ ]:
# sample matrix with nonzero columns only.
sample_nz_col = keep_cols(sample_read_np, nonzero_index)
sample_nz_col

Next we identify all sample points with missing location.

In [ ]:
sample_nz_col.T[1]

In [ ]:
# locatable indices.
keep_row_indices = np.argwhere(~np.isnan(sample_nz_col.T[1])).T[0]
keep_row_indices

In [ ]:
sample_nz = sample_nz_col[keep_row_indices]
sample_nz, sample_nz.shape

Identify the sample points corresonding to the kept rows.

In [ ]:
from opioid_crisis_lib import identify_sample_points
sample_point_id = identify_sample_points(keep_row_indices, ddf_yyyy)
sample_point_id

In [ ]:
sample_point_id.shape

These will be readily available.

In [ ]:
sample_point_id_join = np.array([' '.join(sample_point_id[i]) for i in range(len(sample_point_id))])
sample_point_id_join

### Standardization.

We standardize each feature column $c$: $$\tilde{c} = \dfrac{c - \mathrm{mean}\,(c)}{\mathrm{std}\,(c)}$$

In [ ]:
from opioid_crisis_lib import standardize
sample_standardized = standardize(sample_nz)
sample_standardized

We can imbue this standardized matrix with its original features and identifiers (standardized).

In [ ]:
df_standardized = pd.DataFrame(sample_standardized, index=sample_point_id_join,
                              columns=nonzero_features)
df_standardized

# Principal Component Analysis

We can finally perform PCA...

In [ ]:
np.array(df_standardized)[370]

In [ ]:
from sklearn.decomposition import PCA

k = 80

pca = PCA(n_components=k)
principalComponents = pca.fit_transform(df_standardized.loc[:].values)
principalDf = pd.DataFrame(data = principalComponents,
                           columns = ['principal component {}'.format(_k+1) for _k in range(k)])
principalDf.index = sample_point_id_join

principalDf

Take a look at the explained variances (rounded to 3 decimal places).

In [ ]:
np.round(pca.explained_variance_ratio_, 3)

Explained variances of the first 10 principal components.

In [ ]:
[sum(pca.explained_variance_ratio_[:k+1]) for k in range(20)]

We perform an eigen decomposition of the covariance matrix.

In [ ]:
eigen_values, eigen_vectors = np.linalg.eig(principalDf.cov())
eigen_values

## Visualization

Plot of sample points wrt first two principal components.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 8))
plt.scatter(principalDf['principal component 1'], principalDf['principal component 2'],
           alpha=.1)

Plot first three components.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(8, 8))
ax = Axes3D(fig)

xx = principalDf['principal component 1']
yy = principalDf['principal component 2']
zz = principalDf['principal component 3']

ax.scatter(xx, yy, zz, alpha=1)
ax.set_xlabel('principal component 1')
ax.set_ylabel('principal component 2')
ax.set_zlabel('principal component 3')
plt.show()

So even though the first three principal components only explain approximately 40 percent of the data, it still has enough structure. In fact, by decreasing the opacity of points we see the majority of sample points follow the trajectory of principal component 1.

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = Axes3D(fig)

xx = principalDf['principal component 1']
yy = principalDf['principal component 2']
zz = principalDf['principal component 3']

ax.scatter(xx, yy, zz, alpha=.1)
ax.set_xlabel('principal component 1')
ax.set_ylabel('principal component 2')
ax.set_zlabel('principal component 3')
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = Axes3D(fig)

xx = principalDf['principal component 1']
yy = principalDf['principal component 2']
zz = principalDf['principal component 3']

ax.scatter(xx, yy, zz, alpha=.1)
ax.set_xlabel('principal component 1')
ax.set_ylabel('principal component 2')
ax.set_zlabel('principal component 3')

ax.view_init(10, -10)

plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = Axes3D(fig)

xx = principalDf['principal component 1']
yy = principalDf['principal component 2']
zz = principalDf['principal component 3']

ax.scatter(xx, yy, zz, alpha=.1)
ax.set_xlabel('principal component 1')
ax.set_ylabel('principal component 2')
ax.set_zlabel('principal component 3')

ax.view_init(10, 10)

plt.show()

## Analysis of Principal Components

We can verify that the singular values agree with our eigen decomposition. However, principal components will be obained via SVD due to numerical stability.

In [ ]:
from opioid_crisis_lib import threshold_pass

In [ ]:
u, s, vh = np.linalg.svd(sample_standardized.T, full_matrices=False)
eigen_values_hat = s**2/(3007-1)
eigen_values_hat

The principal components.

In [ ]:
pc_svd = np.matmul(u, np.diag(s))
pc_svd.T[1]

Extract singular values.

In [ ]:
pc_svd = np.matmul(u, np.diag(s))
pc_svd.T, pc_svd.T.shape

We look at each of the singular values and see what is going on.

In [ ]:
from opioid_crisis_lib import pc_explain
pc_explain(0, pc_svd, 50, nonzero_features)

Conclusion (PCA1): as the total population decreases, the total population decreases.

In [ ]:
lab1 = 'Estimate; ANCESTRY - Total population'
lab2 = 'Estimate; FERTILITY - Number of women 15 to 50 years old who had a birth in the past 12 months'
plt.figure(figsize=(8, 8))
plt.scatter(df_standardized[lab1], df_standardized[lab2], alpha=.1)
plt.show()

In [ ]:
lab1 = 'Estimate; ANCESTRY - Total population'
lab2 = 'Buprenorphine'
plt.figure(figsize=(8, 8))
plt.scatter(df_standardized[lab1], df_standardized[lab2], alpha=.1)
plt.show()

In [ ]:
pc_explain(1, pc_svd, 10, nonzero_features)

Conclusion (PCA2): in general, increase in population and married couples decreases use of various drugs.

In [ ]:
lab1 = 'Acetyl fentanyl'
lab2 = 'Fentanyl'
plt.figure(figsize=(8, 8))
plt.scatter(df_standardized[lab1], df_standardized[lab2], alpha=.1)
# plt.ylim(0, 10)
plt.show()

In [ ]:
pc_explain(2, pc_svd, 30, nonzero_features)

These drugs go along with each other.

In [ ]:
lab1 = 'Levorphanol'
lab2 = 'cis-3-methylfentanyl'
plt.figure(figsize=(8, 8))
plt.scatter(df_standardized[lab1], df_standardized[lab2], alpha=1)
# plt.ylim(0, 10)
plt.show()

In reality, the relations implied by the second and third principal components are not very meaningful, because they are mostly influenced by outliers. A more appropriate approach may be to remove those outliers and retry the PCA.

# PCA - Total Drug Reports

In this section we perform PCA with all drug features lumped into one feature, known as `'total drug use'`.

Note that the first 24 features are non-drug related features.

In [ ]:
nonzero_features[24:]

We construct a matrix with 25 features, the first 24 non-drug, and the 25th is the total drug use feature.

In [ ]:
sample_nz.T[24:]

In [ ]:
sampleTotalDrug = np.concatenate((sample_nz.T[:24].T, np.zeros((sample_nz.shape[0], 1))), 1)
sampleTotalDrug.T[24] = np.sum(sample_nz.T[24:], axis=0)
sampleTotalDrug

The last row is the total drug report for a sample observation.

In [ ]:
featuresTotalDrug = np.copy(nonzero_features[:25])
featuresTotalDrug[24] = "Total Drug Reports"
dfTotalDrug = pd.DataFrame(sampleTotalDrug, index=sample_point_id, columns=featuresTotalDrug)
dfTotalDrug

In [ ]:
dfTotalDrugStandardized = pd.DataFrame(standardize(dfTotalDrug),
                                       index=sample_point_id,
                                       columns=featuresTotalDrug)

u, s, vh = np.linalg.svd(dfTotalDrugStandardized.T, full_matrices=False)
pc_svd = np.matmul(u, np.diag(s))
pc_svd.T[0]

In [ ]:
pc_explain(0, pc_svd, 40, featuresTotalDrug)

In [ ]:
lab1 = 'Estimate; ANCESTRY - Total population'
lab2 = 'Total Drug Reports'
plt.figure(figsize=(8, 8))
plt.scatter(dfTotalDrugStandardized[lab1], dfTotalDrugStandardized[lab2], alpha=.1)
plt.show()

In [ ]:
pc_explain(1, pc_svd, 10, featuresTotalDrug)

## Analysis of only Drug Reports

It's possible that the high correlations between socioeconomic factors are "crowding out" the drug use factors. In this case we'll perform analysis on the drugs. This may reveal economic patterns.

In [ ]:
dfDrugReports = df_standardized[nonzero_features[24:]]
u, s, vh = np.linalg.svd(dfDrugReports.T, full_matrices=False)
pc_svd = np.matmul(u, np.diag(s))
pc_svd.T[0]

In [ ]:
pc_explain(0, pc_svd, 30, nonzero_features[24:])

As we see in the principal components: the increased use of one drug seems to be correlated to the increased use of another.

In [ ]:
lab1 = 'Heroin'
lab2 = 'Fentanyl'
plt.figure(figsize=(8, 8))
plt.scatter(dfDrugReports[lab1], dfDrugReports[lab2], alpha=.1)
plt.show()

## Analysis of only Socio-Economic Data

Since we're already here, we might as well take a look at the socio-economic data (without drug factors).

In [ ]:
dfSoc = df_standardized[nonzero_features[:24]]
u, s, vh = np.linalg.svd(dfSoc.T, full_matrices=False)
pc_svd = np.matmul(u, np.diag(s))
pc_svd.T[0]

In [ ]:
pc_explain(0, pc_svd, 50, nonzero_features[:24])

In [ ]:
pc_explain(1, pc_svd, 10, nonzero_features[:24])

Nothing interesting.

## Color-coded maps

Let's identify the points by color: VA blue, OH green, PA orange, KY red, WV black.

We will work with the original dataset. (Doesn't need to be standardized, since it's a visualization)

In [ ]:
df_sample = pd.DataFrame(sample_nz, index=sample_point_id_join,
                              columns=nonzero_features)

In [ ]:
state_initials = np.array([a.split()[1] for a in sample_point_id_join])

vaIndex = list(filter(lambda i: state_initials[i] == 'va', range(len(state_initials))))
ohIndex = list(filter(lambda i: state_initials[i] == 'oh', range(len(state_initials))))
paIndex = list(filter(lambda i: state_initials[i] == 'pa', range(len(state_initials))))
kyIndex = list(filter(lambda i: state_initials[i] == 'ky', range(len(state_initials))))
wvIndex = list(filter(lambda i: state_initials[i] == 'wv', range(len(state_initials))))

states = ['va', 'oh', 'pa', 'ky', 'wv']
stateIndexList = [vaIndex, ohIndex, paIndex, kyIndex, wvIndex]
colorList = ['blue', 'green', 'orange', 'red', 'black']

In [ ]:
plt.figure(figsize=(8, 8))

lab1 = 'Estimate; ANCESTRY - Total population'
lab2 = 'Heroin'

for i in range(5):
    index = stateIndexList[i]
    plt.scatter(df_sample[lab1][index], df_sample[lab2][index], alpha=.1, color=colorList[i])
    
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))

for i in range(5):
    index = stateIndexList[i]
    plt.scatter(principalDf['principal component 1'][index],
                principalDf['principal component 2'][index], alpha=.1, color=colorList[i])
    
plt.show()